## Fidelity Dividend Tool

### Setup
1. Download transaction and activities as a `.csv` file from Fidelity. Recommended filtering for only Dividend Events
2. Move to the `Data/CSV/Fidelity/Dividends` folder
3. Make sure the downloaded csv file ends in `*-Dividends.csv` i.e. `2024-Dividends.csv`

In [ ]:
import os
import pandas as pd

# Configure width of display for pandas data frame
pd.set_option('display.width', 400)
# Configure maximum columns shown
pd.set_option('display.max_columns', 15)
# Do not limit number of rows displayed, allows scroll
pd.set_option("display.max_rows", None)

# Create a folder called Data at the root of this project
base_path = '../Data'
column_converters = {'Run Date': pd.to_datetime,
                     'Account': str,
                     'Action': str,
                     'Symbol': str,
                     'Description': str,
                     'Type': str,
                     'Amount': float
                     }
print_columns = ['Run Date', 'Symbol', 'Amount']

## Read in CSV Files
Read in the CSV files under `Data/Fidelity/Dividends` to combine into a single data frame for further processing.
Clean up by removing the account no.

In [ ]:
# Main Data frame, empty by default
data = pd.DataFrame()

for root, dirs, files in os.walk(base_path):
    for file in files:
        if 'Dividends' in root and '-Dividends.csv' in file:
            file_path = os.path.join(root, file)
            # Read in csv data with the inferred data type and specified columns
            df_raw = pd.read_csv(file_path, converters=column_converters, usecols=lambda cols: cols in column_converters.keys())
            # Remove the Account number - .str is needed because it's working with a Series data type
            df_raw['Account'] = df_raw['Account'].str.rsplit(' ', n=1).str[0]
            # Append to data
            data = pd.concat([data, df_raw], ignore_index=True)
            print(file_path)
            
print(data)

## Dividends Received Grouped By Run Date + Symbol
Filter for only Dividends received and combine the amounts if matching date and symbol

In [ ]:
dividends_data = pd.DataFrame(data)
dividends_received_by_symbol = dividends_data[dividends_data['Action'].str.contains('DIVIDEND RECEIVED', case=False, na=False)]

dividends_received = (dividends_received_by_symbol
                      .groupby(['Run Date', 'Symbol'], as_index=False)
                      .sum())

print(dividends_received[print_columns])

## Dividends Received Grouped By Run Date (Year-Month) + Account
Filter for only Dividends received and combine the amounts if matching date and symbol

In [ ]:
dividends_data = pd.DataFrame(data)
# dividends_data['Run Date'] = pd.to_datetime(dividends_data['Run Date'])
# Copy was explicitly used to avoid the warning when adding the ''Year-Month' that it would affect the original
dividends_received_by_account = dividends_data[dividends_data['Action'].str.contains('DIVIDEND RECEIVED', case=False, na=False)].copy()

dividends_received_by_account.loc[:,'Year-Month'] = dividends_received_by_account['Run Date'].dt.to_period('M')

dividends_received_by_account = (dividends_received_by_account
                                 .groupby(['Year-Month', 'Account'], as_index=False)['Amount']
                                 .sum())
print(dividends_received_by_account[['Year-Month', 'Account', 'Amount']])
